In [ ]:
from fit_param import test

In [ ]:
# case 1: dark matter mass m_chi = 0.05 GeV
param_fits = test([0.05, 29000], cores=-1, repeat=10, plot=False)

In [ ]:
param_fits

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(param_fits)
# plt.show()